<a href="https://colab.research.google.com/github/gioguarnieri/floods_in_sao_paulo/blob/main/Flood_analysis_Graphics_and_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flood analysis: Graphics and statistics
Made by: Giovanni Guarnieri Soares

In this code we plot and make statistics analysis of our street map, in here it is not done any work on the graph itself, only the plottings and tests. The calculations on graphs is in another code which will be released later, when finished and polished.

This notebook uses 3 inputs, the floods dataset (*url_alaga*), the nodes to cross (*url_nodes*) and the graphs metrics (*metrics* data frame).

In [1]:
%%capture
!pip install -U kaleido

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go # plotting module
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from scipy.stats import mannwhitneyu

url_alaga = r'https://raw.githubusercontent.com/gioguarnieri/floods_in_sao_paulo/main/floods.csv?token=GHSAT0AAAAAACA4T26H3QMVUO3L5ZAHG6ROZBJM75A'
url_nodes = r'https://raw.githubusercontent.com/gioguarnieri/floods_in_sao_paulo/main/g4g_nodes.csv?token=GHSAT0AAAAAACA4T26H32V7EYJGGFY55EFUZBJNAPQ'

alaga_df = pd.read_csv(url_alaga, sep = ';')
nodes_df = pd.read_csv(url_nodes)

pd.set_option('display.max_columns', 40)

In [3]:
flood = [] # list to count floods
bg = 1 # beginning month
end = 3 # end month
slice_alaga_df = (alaga_df[(alaga_df["MES"] >= bg) & (alaga_df["MES"] <= end)]["LOCAL_ED"]) # retrieving the sliced localizations
for local in nodes_df["nome_caps"]: #crossing
  try :
    flood.append(slice_alaga_df.value_counts()[local])
  except:
    flood.append(0)

nodes_df["flood"] = flood
nodes_df.to_csv(f"Recorte_TTI_{bg}_{end}.csv") # saving data set

In [4]:
slice_alaga_df = (alaga_df[(alaga_df["MES"] >= bg) & (alaga_df["MES"] <= end)])

In [5]:
alagamentos = slice_alaga_df.loc[slice_alaga_df['LOCAL_ED'].isin(nodes_df["nome_caps"])]

In [6]:
metrics = pd.read_csv("https://raw.githubusercontent.com/gioguarnieri/floods_in_sao_paulo/main/recorte_TTI_data.dat") # recovering the graph metrics calculated before
metrics["Node"] = metrics["Node"] + 1

metrics

,Node,ms,mean_comm,vuln,betw,all_degree,in_degree,out_degree,e_mean,vuln_efi
0,1,7.791411,94.868298,0.003254,0.000000,6,0,6,0.027206,0.003187
1,2,8.205479,74.812881,0.003518,16.000000,6,1,5,0.022684,0.003053
2,3,7.937008,60.768150,0.003879,198.083333,6,2,4,0.019780,0.002732
3,4,9.187500,57.793786,0.002793,15.000000,6,1,5,0.018961,0.002665
4,5,9.320000,48.341717,0.002544,16.000000,5,1,4,0.017764,0.002499
...,...,...,...,...,...,...,...,...,...,...
1208,1209,0.000000,0.000000,0.000000,0.000000,2,2,0,0.000000,0.000000
1209,1210,0.000000,0.000000,0.000000,0.000000,10,10,0,0.000000,0.000000
1210,1211,0.000000,0.000000,0.000000,0.000000,2,2,0,0.000000,0.000000
1211,1212,0.000000,0.000000,0.000000,0.000000,3,3,0,0.000000,0.000000


In [7]:
merge = pd.merge(nodes_df, metrics, how = 'inner', left_on = 'gid', right_on = "Node") 
merge.to_csv("merged_TTI.dat") # merged data set
grava = merge.drop(["gid", "grau", "grau_in", "grau_out", "coef_aglom"], axis=1)
grava.to_csv("TTI_data.csv",index = "F")
alagados = merge[merge["flood"] > 0]
nao_alagados = merge[merge["flood"] == 0]


### Histograms

In [8]:

fig = make_subplots(rows=2, cols=3)
 

fig.add_trace(go.Histogram(x=list(merge["all_degree"]), histnorm='percent', name = "Degree Coefficient"), row=1, col=1)

fig.add_trace(go.Histogram(x=list(merge["ms"]), histnorm='percent', name = "Mean Shortest Path Length"), row=1, col=2)

fig.add_trace(go.Histogram(x=list(merge["betw"]), histnorm='percent', name = "Betweenness Centrality"), row=1, col=3)

fig.add_trace(go.Histogram(x=list(merge["mean_comm"]), histnorm='percent', name = "Mean Communicability Value"), row=2, col=1)

# fig.add_trace(go.Histogram(x=list(merge["vuln"]), histnorm='percent', name = "Vulnerability with Communicability"), row=2, col=2)

fig.add_trace(go.Histogram(x=list(merge["e_mean"]), histnorm='percent', name = "Mean Efficiency Value"), row=2, col=2 )

fig.add_trace(go.Histogram(x=list(merge["vuln_efi"]), histnorm='percent', name = "Vulnerability Coefficient"), row=2, col=3 )


fig.update_layout(
    
    title=f"Histogram to entire data set: Metric value x Percentage",
    # title_x=0.8,
    # xaxis_title="Value",
    # yaxis_title="Percentage",
    legend_title="Metric",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=25,
        # color="black"
    ),
    width=1600, height=900,
)

fig.show()

fig.write_image("all_metrics_histo2.png")
fig.write_image("all_metrics_histo2.pdf")


In [9]:
fig = make_subplots(rows=2, cols=3)
 
fig.add_trace(go.Histogram(x=list(nao_alagados["all_degree"]), histnorm='percent', name = "Degree Coefficient"), row=1, col=1)

fig.add_trace(go.Histogram(x=list(nao_alagados["ms"]), histnorm='percent', name = "Mean Shortest Path Length"), row=1, col=2)

fig.add_trace(go.Histogram(x=list(nao_alagados["betw"]), histnorm='percent', name = "Betweenness Centrality"), row=1, col=3)

fig.add_trace(go.Histogram(x=list(nao_alagados["mean_comm"]), histnorm='percent', name = "Mean Communicability Value"), row=2, col=1)

# fig.add_trace(go.Histogram(x=list(nao_alagados["vuln"]), histnorm='percent', name = "Vulnerability with Communicability"), row=2, col=2)

fig.add_trace(go.Histogram(x=list(nao_alagados["e_mean"]), histnorm='percent', name = "Mean Efficiency Value"), row=2, col=2 )

fig.add_trace(go.Histogram(x=list(nao_alagados["vuln_efi"]), histnorm='percent', name = "Vulnerability Coefficient"), row=2, col=3 )


fig.update_layout(
    
    title=f"Histogram to non-flooded streets: Metric value x Percentage",
    # title_x=0.8,
    # xaxis_title="X Axis Title",
    # yaxis_title="Data",
    legend_title="Metric",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=25,
        # color="black"
    ),
    width=1600, height=900,
)

fig.show()

fig.write_image("all_metrics_histo_non_flooded.png")
fig.write_image("all_metrics_histo_non_flooded.pdf")

In [10]:
fig = make_subplots(rows=2, cols=3)
 
fig.add_trace(go.Histogram(x=list(alagados["all_degree"]), histnorm='percent', name = "Degree Coefficient"), row=1, col=1)

fig.add_trace(go.Histogram(x=list(alagados["ms"]), histnorm='percent', name = "Mean Shortest Path Length"), row=1, col=2)

fig.add_trace(go.Histogram(x=list(alagados["betw"]), histnorm='percent', name = "Betweenness Centrality"), row=1, col=3)

fig.add_trace(go.Histogram(x=list(alagados["mean_comm"]), histnorm='percent', name = "Mean Communicability Value"), row=2, col=1)

# fig.add_trace(go.Histogram(x=list(alagados["vuln"]), histnorm='percent', name = "Vulnerability with Communicability"), row=2, col=2)

fig.add_trace(go.Histogram(x=list(alagados["e_mean"]), histnorm='percent', name = "Mean Efficiency Value"), row=2, col=2 )

fig.add_trace(go.Histogram(x=list(alagados["vuln_efi"]), histnorm='percent', name = "Vulnerability Coefficient"), row=2, col=3 )




fig.update_layout(
    
    title=f"Histogram to flooded streets: Metric value x Percentage",
    # title_x=0.8,
    # xaxis_title="X Axis Title",
    # yaxis_title="Data",
    legend_title="Metric",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=25,
        # color="black"
    ),
    width=1600, height=900,
)

fig.show()

fig.write_image("all_metrics_histo_flooded.png")
fig.write_image("all_metrics_histo_flooded.pdf")

In [11]:
metrics = ["all_degree", "ms", "mean_comm", "e_mean", "betw", "vuln_efi"]
for i in metrics:
  print(f"""
  {i}
    Flooded:
  mean - {alagados[i].mean()}
  median - {alagados[i].median()}
  std - {alagados[i].std()}
    Non-Flooded:
  mean - {nao_alagados[i].mean()}
  median - {nao_alagados[i].median()}
  std - {nao_alagados[i].std()}
  """)


  all_degree
    Flooded:
  mean - 5.075675675675676
  median - 5.0
  std - 1.1631585104056217
    Non-Flooded:
  mean - 4.866731517509727
  median - 5.0
  std - 1.3467629400341032
  

  ms
    Flooded:
  mean - 2.4823926769898703
  median - 1.9230769230769231
  std - 2.1523417372067803
    Non-Flooded:
  mean - 3.538848222625737
  median - 3.294117647058824
  std - 2.4738562274901996
  

  mean_comm
    Flooded:
  mean - 22.211258029138325
  median - 16.375000026077032
  std - 24.005408970372574
    Non-Flooded:
  mean - 24.362136179255018
  median - 18.922025668139582
  std - 21.992393726176726
  

  e_mean
    Flooded:
  mean - 0.00590826321622075
  median - 0.0042629262926292
  std - 0.005718989853624422
    Non-Flooded:
  mean - 0.00724069705246212
  median - 0.00597890146157465
  std - 0.0059351096783116725
  

  betw
    Flooded:
  mean - 97.75090051090051
  median - 29.666666666666664
  std - 220.689503181443
    Non-Flooded:
  mean - 152.6683690714819
  median - 61.2833333333

### Violin plot

In [12]:
metric = "all_degree"
fig = go.Figure()

fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Degree",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Degree",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))


fig.update_layout(
    title=f"Violin plot: Degree",
    # xaxis_title="X Axis Title",
    yaxis_title="All Degree",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.png")

In [13]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 1.132908666270815, kurt non-flooded 2.285071080581245
skew flooded: 0.20739526078180848, skew non-flooded 0.6798360296282608


In [14]:
metric = "in_degree"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: In Degree",
    # xaxis_title="X Axis Title",
    yaxis_title="Data",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.pdf")

In [15]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: -0.41915356770768497, kurt non-flooded 1.6554345752235866
skew flooded: 0.3330739035745255, skew non-flooded 0.6894648021309411


In [16]:
metric = "out_degree"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: Out degree",
    # xaxis_title="X Axis Title",
    yaxis_title="Data",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.pdf")

In [17]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 0.9012475005938527, kurt non-flooded 1.3161004292362488
skew flooded: 0.7047259603987059, skew non-flooded 0.7817158900872411


In [18]:
metric = "ms"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: Mean Shortest Path",
    # xaxis_title="X Axis Title",
    yaxis_title="Mean Shortest Path",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.png")

In [19]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 0.51742267253905, kurt non-flooded -0.3583817658007602
skew flooded: 1.0749601096553687, skew non-flooded 0.5622712761264559


In [20]:
metric = "vuln"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: Vulnerability with Communicability",
    # xaxis_title="X Axis Title",
    yaxis_title="Vulnerability with Communicability",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.png")

In [21]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 7.403406119088176, kurt non-flooded 10.280244575550862
skew flooded: 1.8590690672053753, skew non-flooded 2.1403235409835157


In [22]:
metric = "vuln_efi"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: Vulnerability",
    # xaxis_title="X Axis Title",
    yaxis_title="Vulnerability",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.png")

In [23]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 19.332354382522162, kurt non-flooded 20.89409190322097
skew flooded: 3.8067405201331024, skew non-flooded 3.5489674316541255


In [24]:
metric = "mean_comm"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: Mean Commucability",
    # xaxis_title="X Axis Title",
    yaxis_title="Mean Commucability",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.png")

In [25]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 8.808571997464588, kurt non-flooded 3.6738769167336436
skew flooded: 2.3486043893895134, skew non-flooded 1.654405819916574


In [26]:
metric = "e_mean"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: Mean Efficiency",
    # xaxis_title="X Axis Title",
    yaxis_title="Mean Efficiency",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.png")

In [27]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 3.7653519017743573, kurt non-flooded 3.0420353046191604
skew flooded: 1.6841360188896655, skew non-flooded 1.4390137993831131


In [28]:
print(f"""
Description flooded: {alagados[metric].describe()}

Description non-flooded: {nao_alagados[metric].describe()}
""")


Description flooded: count    185.000000
mean       0.005908
std        0.005719
min        0.000000
25%        0.001788
50%        0.004263
75%        0.008834
max        0.032138
Name: e_mean, dtype: float64

Description non-flooded: count    1028.000000
mean        0.007241
std         0.005935
min         0.000000
25%         0.002811
50%         0.005979
75%         0.010540
max         0.034195
Name: e_mean, dtype: float64



In [29]:
metric = "betw"
fig = go.Figure()
 
fig.add_trace(go.Violin(y=alagados[metric], box_visible=True, line_color='black', side = "negative",
                               meanline_visible=True, fillcolor='Yellow', opacity=0.6, x0 = "Flood data",
                        legendgroup='Flooded', scalegroup='Flooded', name='Flooded'))

fig.add_trace(go.Violin(y=nao_alagados[metric], box_visible=True, line_color='black', side = "positive",
                               meanline_visible=True, fillcolor='Purple', opacity=0.6, x0 = "Flood data",
                        legendgroup='Non-Flooded', scalegroup='Non-Flooded', name='Non-Flooded'))
fig.update_layout(
    title=f"Violin plot: Betweenness",
    # xaxis_title="X Axis Title",
    yaxis_title="Betweenness",
    legend_title="Type",
    showlegend=True,
    font=dict(
        family="Courier New, monospace",
        size=18,
        # color="black"
    ),
    width=800, height=900,
)
fig.show()

fig.write_image(metric+"_violin.png")

In [30]:
print(f"Kurt flooded: {alagados[metric].kurtosis()}, kurt non-flooded {nao_alagados[metric].kurtosis()}")
print(f"skew flooded: {alagados[metric].skew()}, skew non-flooded {nao_alagados[metric].skew()}")

Kurt flooded: 26.04443658803377, kurt non-flooded 22.45552380053281
skew flooded: 4.757338789015276, skew non-flooded 3.87496384280636


### Mann-Withney U test

In [31]:
alagados.drop(['gid', 'id', 'length', 'dir', 'municipio', 'distrito', 'bairro',
       'nomesigla', 'tipo', 'nometit', 'nomeprep', 'name', 'start_left',
       'end_left', 'start_righ', 'end_right', 'parity', 'left_zip',
       'right_zip', 'cep_e', 'cep_d', 'extensao_m', 'nome_caps', 'onibus_msp',
       'dist_cen_m', 'dist_cen_d', 'cdlog_cem', 'grau', 'grau_in', 'grau_out',
       'coef_aglom', 'Node', "flood"], inplace=True, axis=1)
nao_alagados.drop(['gid', 'id', 'length', 'dir', 'municipio', 'distrito', 'bairro',
       'nomesigla', 'tipo', 'nometit', 'nomeprep', 'name', 'start_left',
       'end_left', 'start_righ', 'end_right', 'parity', 'left_zip',
       'right_zip', 'cep_e', 'cep_d', 'extensao_m', 'nome_caps', 'onibus_msp',
       'dist_cen_m', 'dist_cen_d', 'cdlog_cem', 'grau', 'grau_in', 'grau_out',
       'coef_aglom', 'Node', "flood"], inplace=True, axis = 1)

<ipython-input-31-addf05903220>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-31-addf05903220>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:

plist = []
ulist = []
for i in alagados.columns:
  U1, p = mannwhitneyu(nao_alagados[i], alagados[i], alternative="two-sided")
  ulist.append(U1)
  plist.append(p)

df_mwu = pd.DataFrame({"Metric" : alagados.columns, "score" : ulist, "p": plist})
df_mwu

,Metric,score,p
0,ms,119840.5,1.655680e-08
1,mean_comm,104526.5,3.140590e-02
2,vuln,106256.5,1.089051e-02
3,betw,113595.5,2.350240e-05
4,all_degree,83678.5,7.075444e-03
5,in_degree,90239.0,2.562922e-01
6,out_degree,93806.0,7.638055e-01
7,e_mean,110245.5,5.478904e-04
8,vuln_efi,110362.5,4.967760e-04
